# PoseButcher Example

> _A good butcher always trims the fat_

PoseButcher is a tool for categorising and segmenting virtual hits with reference to experimental protein structures and (fragment) hits.

This notebook is demonstrates the features of PoseButcher with crystal structures from the [Enteroviral 2A protease](https://fragalysis.diamond.ac.uk/viewer/react/preview/target/A71EV2A/tas/lb18145-1).

### This example at a glance

1. Create the **butcher**:

2. **Chop** up a posed virtual hit (rdkit `Mol` with a conformer):

3. **Tag** a compound based on its pocket occupancy and clashes:

4. **Explore** the expansion opportunities from a given atom in a virtual hit

5. (Coming soon) **Trim** a parts of a compound that clash with a protein or leave the crystal

6. (Coming soon) **Score** how well a virtual hit recapitulates shape and colour of the fragment bolus

## Imports

In [ ]:
from posebutcher import PoseButcher
from rdkit.Chem import PandasTools
import pandas as pd
import molparse as mp
import plotly.express as px

## 1. Create the butcher

### Specify the protein template

This must be a `str` or `pathlib.Path` to a reference protein structure. Non-protein atoms will automatically be ignored.

In [ ]:
protein = 'test_data/2a_hits/A71EV2A-x0310_0A_bound.pdb'

### Specify the reference fragments

In [ ]:
hits = 'test_data/2a_fragments.sdf'

### Define the catalytic/allosteric pockets

Defining the pockets can be tricky, I suggest you use an external tool (PyMOL/Fragalysis) to pick the correct atoms.

For now only spherical pockets are supported.

In any case, pockets should be a `dict` with `str` keys and `dict` values. Below, spherical pockets are defined at the centre of mass of several atoms with a radius defined by the average distance from CoM to the atoms or a given value:

In [ ]:
pockets = {
    "P1": dict(type='sphere', atoms=['GLY 127 O', 'PRO 107 CG', 'CYS 110 SG'], radius='mean'),
    "P2": dict(type='sphere', atoms=['VAL 84 CG1', 'TYR 90 CD2', 'SER 87 CB'], radius='mean'),
    "P1'": dict(type='sphere', atoms=['GLU 88 CB', 'PRO 107 CB', 'HIS 21 CD2'], radius='mean'),
    "P2'": dict(type='sphere', atoms=['PRO 107 CB', 'LEU 22 CD1'], shift=[0, 0, 0], radius='mean'),
    "P3": dict(type='sphere', atoms=['GLY 127 O', 'GLU 85 CB'], radius=4),
    "P4": dict(type='sphere', atoms=['LEU 98 CD2'], radius=5),
    "P5": dict(type='sphere', atoms=['ASN 129 ND2', 'ASN 129 ND2', 'ILE 82 CG2'], radius=4),
    "P6": dict(type='sphere', atoms=['ILE 82 CG2'], shift=[-1,0,1], radius=4),
}

N.B. Other options for radius are: 'min' or 'max' and shift can be used to manually move the centre of the pocket.

### Create the butcher

In [ ]:
butcher = PoseButcher(protein, hits, pockets)

### Render the resulting complex

This will open an Open3d viewer of the various meshes. Useful features:

* `Scene / Show Axis` Show the coordinate axes (x=red, y=blue, z=green)
* `Geometries` Toggle visibility of different meshes

In [ ]:
butcher.render()

## 2. Chop up a posed de novo compound

In this case we are loading an SD file containing posed molecules

In [ ]:
mol_df = PandasTools.LoadSDF('test_data/2a_compounds.sdf')
mol = mol_df.iloc[45]['ROMol']
mol._Name = mol_df.iloc[45]['ID']

Now we **chop** the molecule by pocket or protein/solvent clashes

In [ ]:
result = butcher.chop(mol, draw='3d')

the result will be a dictionary with atom indices as keys:

In [ ]:
result

The compound can also be visualised in 3d or not at all with the option `draw=False`:

In [ ]:
butcher.chop(mol, draw='3d');

If your ligand is an elaboration or expansion of a known **base** (parent) compound you can consider only the novel material:

In [ ]:
butcher.chop(mol, base='C[C@H](NC(=O)/C(C#N))C(=O)Nc1cc[nH]n1')

## 3. Get a compound's tags

If you want to use pockets to tag a compound but care which atoms use `butcher.tag`:

In [ ]:
butcher.tag(mol)

In [ ]:
butcher.tag(mol, pockets_only=True)

The familiar `draw` options of `'2d'` and `'3d'` are still available, as is the `base` argument.

## 4. Explore expansion vectors

Posebutcher can categorise and score expansion opportunities from atoms in a ligand.

`butcher.explore` can cast a ray from a given atom index:

In [ ]:
result = butcher.explore(mol, origin = 23)

N.B. you can use `mp.rdkit.draw_flat` to see the atom indices labelled:

In [ ]:
mp.rdkit.draw_flat(mol, indices=True)

The output from `butcher.explore` is a dictionary using the familiar classification from `butcher.chop`. 

e.g. `('BAD', 'protein clash')`. Important keys to note are:

* 'origin': classification of the start of the vector
* 'intersections': a dictionary of intersections the vector makes. The keys are distances in Angstrom. The values are the classification tuples.
* 'first_intersection_distance': the distance at which the first intersection is made.
* 'last_intersection_distance': the distance at which the final intersection is made (protein/solvent).
* 'new_pocket': True if the vector explores a new pocket
* 'destination': description of the end of the vector
* 'max_atoms_added': an estimate for the number of heavy atoms that can be added


In [ ]:
result

`butcher.explore` can also explore all vectors in an atom (atoms bonded to fewer than 3 other atoms)

In [ ]:
result = butcher.explore(mol)

The result is a list of the single origin outputs (dictionaries).

It may be useful to view the results as a DataFrame or with plotly:

In [ ]:
pd.DataFrame(result)

In [ ]:
px.bar(result, color='new_pocket', x='atom_index', y='last_intersection_distance')